In [1]:
import pandas as pd
import os

# 设定文件夹路径
folder_path = os.getcwd()

# 列出所有 CSV 文件
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 读取并合并文件
df_list = [pd.read_csv(os.path.join(folder_path, file)) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# 保存合并后的 DataFrame 到新的 CSV 文件
combined_df.to_csv('combined_file.csv', index=False)

In [3]:
req = pd.read_csv('combined_file.csv')
req.drop(req.columns[0], axis=1, inplace=True)
req.insert(0, "ReqID", range(1, len(req) + 1))
req.to_csv('combined_file.csv', index=False)